## Bounded Boxes

**Observations**  
-Nombre de datasets par département  
-Nombre de dataset par superficie de bounded box suivant 4 catégories : intra-départemental, départemental, régional, supranational  

**Traitements**  
-Données filtrées sur le territoire métropolitain (zones terrestres) par jointure spatiale des centroides des bounded boxes avec les géométries des départements métropolitains

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
from pathlib import Path

import plotly
import plotly.express as px

from utils.utils import wrapper_engine, compute_areas_km2
engine = wrapper_engine('config.ini')

from utils import custom_settings
from itables import show
custom_settings.apply_itable_custom_settings()

In [ ]:
# Load data
with engine.connect() as connection:    
    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )
    
    package = pd.read_sql_table(
        table_name="package", 
        con=connection,
        schema="public",
        )
    
package_extra = package_extra[package_extra.key.isin(["bbox", "crs"])].sort_values(by=["package_id", "key"])
package_extra = pd.pivot_table(package_extra, index=["package_id"], values=["value"], columns=["key"], aggfunc='first')
package_extra = package_extra.droplevel(0, axis=1).reset_index().rename_axis(columns=None)

**Nombre de datasets avec bounded boxes**

In [ ]:
show(package_extra[["package_id","bbox"]].rename(columns={"packaged_id":"datasets"}).describe())

In [ ]:
### WARNING : bounded box coordinates are supposedly projected in EPSG:4326, not to be confused with datasets ESPG metadatas

mainland_data = package_extra.dropna(subset=["bbox"]).copy(deep=True)
mainland_data["bbox"] = mainland_data["bbox"].apply(lambda x: shapely.wkt.loads(x))
mainland_data = gpd.GeoDataFrame(mainland_data, geometry="bbox", crs="EPSG:4326")

mainland_data["centroid"] = mainland_data["bbox"].apply(lambda x: x.centroid)

In [ ]:
admin_express_data = Path(
    r"data/ADMIN-EXPRESS-COG-CARTO_3-0__SHP_WGS84G_FRA_2021-05-19/ADMIN-EXPRESS-COG-CARTO/1_DONNEES_LIVRAISON_2021-05-19/ADECOGC_3-0_SHP_WGS84G_FRA/"
)

#### Read region geometries - Default EPSG: 3857 (pseudo mercator)
regions = gpd.read_file(admin_express_data.joinpath("REGION.shp"))

# Exclude oversea territories
regions = regions[regions["INSEE_REG"].isin([str(int(x)) for x in range(10, 100)])]
# Projection in meter for areas comparison: https://epsg.io/2154
regions["bounded_box"] = regions["geometry"].envelope
regions["area_km2"] = compute_areas_km2(regions["bounded_box"].to_crs("2154"))

In [ ]:
#### Read departments geometries - Default EPSG: 3857 (pseudo mercator)
departments = gpd.read_file(admin_express_data.joinpath("DEPARTEMENT.shp"))

# Exclude oversea territories
departments = departments[departments["INSEE_DEP"].isin([str(int(x)).zfill(2) for x in range(0, 100)] + ['2A', '2B'])]
## Projection in meter for areas comparison: https://epsg.io/2154
departments["bounded_box"] = departments["geometry"].envelope
departments["area_km2"] = compute_areas_km2(departments["bounded_box"].to_crs("2154"))
departments.to_crs("2154", inplace=True)

In [ ]:
# Filter on metropolitan data
metropolitan_data = gpd.sjoin(
    departments[["ID", "NOM", "geometry"]].to_crs(4326),
    mainland_data.set_geometry("centroid"),
    predicate="contains")
metropolitan_data["area_km2"] = compute_areas_km2(metropolitan_data["bbox"].to_crs("3035"))

In [ ]:
metropolitan_data_count = metropolitan_data.groupby("ID", as_index=False).agg(
    {
        "NOM": "first",
        "geometry": "first",
        "bbox": "count"
    }
).set_geometry("geometry").rename(columns={"bbox": "count"}).set_crs(epsg=4326).sort_values(by="count", ascending=False)

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
fig = px.bar(metropolitan_data_count,
             x='NOM', 
             y='count',
             color_discrete_sequence=['#000091'],
            labels={"NOM": "Département",
                    "count": "Nombre"})
fig.add_hline(
    y=int(metropolitan_data_count["count"].median()), 
    line_width=3, 
    line_dash="dash", 
    line_color="red",
    annotation_text="médiane",
    annotation_textangle = 0)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
plotly.offline.iplot(fig)

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
fig = px.choropleth(
                    metropolitan_data_count, 
                    geojson=metropolitan_data_count["geometry"],
                    locations=metropolitan_data_count.index,
                    color='count',
                    color_continuous_scale="Turbo",
                    hover_name="NOM",
                    projection="mercator"
                  )
fig.update_geos(fitbounds="locations", 
                visible=False,)
fig.update_traces(marker_line_width=0)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
plotly.offline.iplot(fig)

In [ ]:
departments_range = range(
    departments["area_km2"].min().astype(int), 
    max(departments["area_km2"].max(), regions["area_km2"].min()).astype(int))

regions_range = range(
    max(departments["area_km2"].max(), regions["area_km2"].min()).astype(int), 
    regions["area_km2"].max().astype(int))

supra_regional = range(
    regions["area_km2"].max().astype(int),
    regions["area_km2"].sum().astype(int))

metropolitan_data["category"] = metropolitan_data["area_km2"].apply(lambda x:
    "01_Intra-départemental" if (int(x) < departments["area_km2"].min().astype(int)) 
    else "02_Départemental" if (int(x) in departments_range)
    else "03_Régional" if (int(x) in regions_range)
    else "04_Suprarégional" if (int(x) in supra_regional)
    else "05_Supranational" if (int(x) > regions["area_km2"].sum().astype(int)) 
    else None)

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
metropolitan_data_category = metropolitan_data.groupby("category", as_index=False).agg(
    {
        "bbox": "count",
    }
).rename(columns={"bbox":"count"})
fig = px.bar(metropolitan_data_category, 
             x='category', 
             y='count',
             color_discrete_sequence=['#000091'],
             labels={"category": "Catégorie",
                    "count": "Nombre"}
             )
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
plotly.offline.iplot(fig)